In [1]:
import json

from typing import List, Tuple

from cjw.aistory.utilities.GptPortal import GptPortal
import os


In [8]:
key = os.environ.get("OPENAI_KEY")
os.environ["OPENAI_API_KEY"] = key
gpt = GptPortal.of(key)
conversation=[]

In [10]:

def addContents(messages: List[dict], contents: str | List[str], role: str = "user"):
    if isinstance(contents, str):
        contents = [contents]

    messages.append({
        "role": role,
        "content": '\n'.join(contents)
    })


In [24]:
tableDescription = """
1. CustomerProfile
- id
- name
- gender (optional)
- age (optional)
- location (optional)

2. Purchases
- id
- customer_id
- product_id
- purchase_price
- quantity
- purchase_time

3. Products
- id
- name
- attributes
- description
"""

In [25]:
systemPrompt = f"""
Consider the following tables and their column names in a database:

{tableDescription}

We are to create a product recommender for a customer.
"""

In [26]:
addContents(conversation, systemPrompt, "system")

In [27]:
addContents(conversation, "What should I consider with this problem?")

In [28]:
conversation

[{'role': 'system',
  'content': '\nConsider the following tables and their column names in a database:\n\n\n1. CustomerProfile\n- id\n- name\n- gender (optional)\n- age (optional)\n- location (optional)\n\n2. Purchases\n- id\n- customer_id\n- product_id\n- purchase_price\n- quantity\n- purchase_time\n\n3. Products\n- id\n- name\n- types\n- description\n\n\nWe are to create a product recommender for a customer.\n'},
 {'role': 'user', 'content': 'Write a python code for it.'},
 {'role': 'system',
  'content': '\nConsider the following tables and their column names in a database:\n\n\n1. CustomerProfile\n- id\n- name\n- gender (optional)\n- age (optional)\n- location (optional)\n\n2. Purchases\n- id\n- customer_id\n- product_id\n- purchase_price\n- quantity\n- purchase_time\n\n3. Products\n- id\n- name\n- types\n- description\n\n\nWe are to create a product recommender for a customer.\n'},
 {'role': 'user', 'content': 'What should I considered with this problem?'},
 {'role': 'system',
  

In [29]:
response = await gpt.chatCompletion(conversation)

In [30]:
print(response["content"])

To create a product recommender system, there are several things you need to think about:

1. Customer profile: This is your base for personalizing the product recommendations. You can analyze your customer's age, gender, and location to wishlist similar products that other users with the same profile have bought.

2. Purchase history: Analyzing the purchase history will give us the best idea of our customer's preferences. Identify the products or product types they have purchased the most, and recommend similar items.

3. Product details: The details of the products can help recommend other products of a similar type or in the same category. Consider the types and the descriptions of frequently purchased items.

4. Collaborative filtering: Consider what similar customers have bought. If a group of customers have bought similar items to this customer, and also bought another item, that item would be a good recommendation.

You might also want to include a time factor, such as recommend

In [31]:
addContents(conversation,
            [
                "I'll use collaborative filtering, considering both the product attributes and customer purchase history.",
                "The tables are in Parquet format.  Please write a PySpark ETL code that prepares the training data."
            ])
response = await gpt.chatCompletion(conversation)

In [32]:
print(response["content"])

Sure, here is a simple PySpark ETL code that reads the Parquet files, processes the data, and prepares it for modeling.

```python
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Product Recommender").getOrCreate()

# Load data
customerProfileDF = spark.read.parquet("path/to/customer_profile.parquet")
purchasesDF = spark.read.parquet("path/to/purchases.parquet")
productsDF = spark.read.parquet("path/to/products.parquet")

# Options to handle missing values would be either dropping the rows or filling them with a specified value. 
# Another way would be to use an imputer to fill missing values based on strategy such as mean or median, 
# however in this scenario we will drop them for simplicity.
customerProfileDF = customerProfileDF.na.drop()
purchasesDF = purchasesDF.na.drop()
productsDF = productsDF.na.drop()

# Data Transformation
# 1. Convert 'gender' column from string to numeric values (male:0, female:1) 
customerProfileDF = customerProfileDF.withColumn